In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
!pip install datasets
!pip install konlpy

# 데이터 로드 및 전처리 함수 정의

In [ ]:
import os
import json

def load_json_dataset(file_path):
    # 하나의 JSON 파일을 열어 문서별로 text, summary 정보를 추출
    with open(file_path, "r", encoding="utf-8") as f:
        data = json.load(f)
    examples = []
    for doc in data["documents"]:
        sentences = []
        # "text"는 중첩 리스트 형태이므로 내부의 모든 sentence를 추출
        for sublist in doc["text"]:
            for item in sublist:
                sentences.append(item.get("sentence", ""))
        full_text = " ".join(sentences)
        # abstractive 요약은 첫번째 항목 사용 (없으면 빈 문자열)
        summary = doc["abstractive"][0] if doc["abstractive"] else ""
        examples.append({
            "text": full_text,
            "summary": summary,
        })
    return examples

# 기본 경로 설정
base_path = "/content/drive/Shareddrives/스프린트(AI) 드라이브/트랙 Master 폴더/스프린트 미션 및 모범답안/data/summarization/"

# law 관련 파일 하나만 불러오기
train_file = "train_original_editorial.json"
valid_file = "valid_original_editorial.json"

train_examples = load_json_dataset(os.path.join(base_path, train_file))
valid_examples = load_json_dataset(os.path.join(base_path, valid_file))

print("Train examples:", len(train_examples))
print("Validation examples:", len(valid_examples))

Train examples: 56760
Validation examples: 7008


# Hugging Face Dataset 변환 및 전처리

In [ ]:
from transformers import BartTokenizerFast

# Fast Tokenizer 사용
checkpoint = "gogamza/kobart-base-v1"
tokenizer = BartTokenizerFast.from_pretrained(checkpoint)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels will be overwritten to 2.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'PreTrainedTokenizerFast'. 
The class this function is called from is 'BartTokenizerFast'.


In [ ]:
from datasets import Dataset, DatasetDict
from transformers import BartTokenizer

# train_examples와 valid_examples로 Dataset 객체 생성
train_dataset = Dataset.from_list(train_examples)
valid_dataset = Dataset.from_list(valid_examples)

# DatasetDict 형태로 통합 (훈련/검증)
dataset = DatasetDict({
    "train": train_dataset,
    "validation": valid_dataset
})

In [ ]:
from datasets import load_dataset, Dataset, DatasetDict
from transformers import BartTokenizerFast, DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer)


model_name = "gogamza/kobart-base-v1"
tokenizer = BartTokenizerFast.from_pretrained(model_name)

def tokenize_function(example):
    model_inputs = tokenizer(
        example["text"],
        max_length=512,
        truncation=True
    )
    labels = tokenizer(
        text_target=example["summary"],
        max_length=256,
        truncation=True
    )
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_datasets = dataset.map(tokenize_function, batched=True)
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer)

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels will be overwritten to 2.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'PreTrainedTokenizerFast'. 
The class this function is called from is 'BartTokenizerFast'.


Map:   0%|          | 0/56760 [00:00<?, ? examples/s]

Map:   0%|          | 0/7008 [00:00<?, ? examples/s]

# 파인튜닝 설정 및 학습

In [ ]:
from transformers import BartForConditionalGeneration, TrainingArguments, Trainer

model = BartForConditionalGeneration.from_pretrained(checkpoint)
training_args = TrainingArguments(output_dir="test-trainer", report_to="none")

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

trainer.train()

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels will be overwritten to 2.
<ipython-input-14-b1fb5abe75e3>:6: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss
500,2.468100
1000,2.284800
1500,2.248200
2000,2.204900
2500,2.194500
3000,2.172200
3500,2.156400
4000,2.143100
4500,2.139000
5000,2.106100


/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:3353: UserWarning: Moving the following attributes in the config to the generation config: {'forced_eos_token_id': 1}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


TrainOutput(global_step=21285, training_loss=1.8276096751719595, metrics={'train_runtime': 2916.9206, 'train_samples_per_second': 58.377, 'train_steps_per_second': 7.297, 'total_flos': 5.18182861787136e+16, 'train_loss': 1.8276096751719595, 'epoch': 3.0})

# 파인튜닝된 모델로 pipeline 추론 테스트

In [ ]:
from transformers import pipeline

# summarization pipeline 생성 (모델과 토크나이저가 이미 정의되어 있다고 가정)
summarizer = pipeline("summarization", model=model, tokenizer=tokenizer)

# 검증 데이터셋의 첫 10개 예시에 대해 요약 결과 출력
for i in range(10):
    sample_text = tokenized_datasets["validation"][i]["text"]
    prompt = f"원문:\n{sample_text}\n요약:"
    print(f"예시 {i+1}:")
    print(prompt)

    # 요약 결과 생성 (필요한 max/min length는 상황에 맞게 조정)
    result = summarizer(prompt, max_length=130, min_length=30, do_sample=False)
    print(result[0]['summary_text'])
    print("-" * 50)

Device set to use cuda:0


예시 1:
원문:
더불어민주당 이해찬 대표가 30 일 오후 국회에서 기자간담회를 열고 조국 전 법무부 장관 사태와 관련해 "국민 여러분께 매우 송구하다"고 밝혔다. 더불어민주당 이해찬 대표가 30 일 기자간담회를 열고 '조국 사태'와 관련, "국민 여러분께 매우 송구하다"는 입장을 밝혔다. 이 대표는 "검찰 개혁이란 대의에 집중하다 보니, 국민 특히 청년이 느꼈을 불공정에 대한 상대적 박탈감, 좌절감을 깊이 있게 헤아리지 못했다"며 "여당 대표로서 무거운 책임감을 느낀다"고 머리를 숙였다. 조국 전 법무부 장관이 14 일 사퇴한 이후 이 대표가 당 안팎의 쇄신 요구에 대해 입장을 표명한 것은 이번이 처음이다. 청와대와 여당은 '조국 정국'을 거치며 분출된 '공정'과 '정의'의 민심을 받들어 검찰 개혁에 매진하겠다면서도 두 달간 극심한 분열과 갈등을 초래한데 대해선 진지하게 성찰하는 모습을 보이지 않았다. 그나마 초선인 이철희 의원이 "당이 대통령 뒤에 비겁하게 숨어 있었다"고 비판했고, 표창원 의원은 "책임을 느끼는 분들이 각자 형태로 그 책임감을 행동으로 옮겨야 할 때"라고 지적했다. 뒤늦게나마 이 대표가 자성의 목소리를 내긴 했으나 당 안팎의 쇄신 요구에 어떻게 응할지 구체적 플랜을 제시하지 못해 여전히 안이하다는 지적도 나온다. 이 대표는 28 일 윤호중 사무총장을 단장으로 하는 총선기획단을 발족했고 조만간 인재영입위원회도 출범시킬 계획이라고 밝혔다. 이 대표는 "민주당의 가치를 공유하는 참신한 인물을 영입해 준비된 정책과 인물로 승부하겠다"고 다짐했다. 하지만 당 일각에선 "총선기획단장을 비롯한 당직 인선부터 쇄신 의지를 보여야 한다"는 비판의 목소리가 나온다. 무조건 물러나는 게 능사는 아니지만 국정 혼선을 초래한 데 대해 당 지도부가 겸허하게 책임지는 모습을 보이는 게 쇄신의 출발점이 돼야 한다는 지적도 있다. 선거는 대중의 이해와 요구를 잘 대표하는 정치인을 뽑는 행위다. 민생을 외면하며 낡은 이념과 진영 싸움에 매몰된 구시대 인물들을 과감히 물갈